In [1]:
import torch
import torch.nn as nn

In [2]:
from embedding_players import get_embeddings

# Device

In [3]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


# Filepaths

In [4]:
BBB_filepath = "C:/Users/lachl/Google Drive/Uni Year 5/git_stuff/STAT3007Project/csv_files/cricsheet_data_partial.csv"

# Import embeddings

In [5]:
#, bowl_at_ball, model, results, pitch
bat_at_ball, bowl_at_ball, model, results, pitch, df = get_embeddings(raw_data_file_name=BBB_filepath,
                                                                      batter_embed_size =10, 
                                                                      bowler_embed_size=10, 
                                                                      num_it = 500)

C:\Users\lachl\Google Drive\Uni Year 5\git_stuff\STAT3007Project\embedding_players.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ret_val[wicket_title] = 1 * (ret_val["wicket_kind"] == wicket_title)


Epoch 0: train loss: 0.6707425117492676
Epoch 1: train loss: 0.6676708459854126
Epoch 2: train loss: 0.6646226644515991
Epoch 3: train loss: 0.6615965366363525
Epoch 4: train loss: 0.658591091632843
Epoch 5: train loss: 0.655604898929596
Epoch 6: train loss: 0.6526370644569397
Epoch 7: train loss: 0.6496859788894653
Epoch 8: train loss: 0.6467506289482117
Epoch 9: train loss: 0.6438298225402832
Epoch 10: train loss: 0.6409225463867188
Epoch 11: train loss: 0.6380277276039124
Epoch 12: train loss: 0.6351441740989685
Epoch 13: train loss: 0.6322711110115051
Epoch 14: train loss: 0.6294073462486267
Epoch 15: train loss: 0.6265521049499512
Epoch 16: train loss: 0.6237043738365173
Epoch 17: train loss: 0.6208631992340088
Epoch 18: train loss: 0.6180277466773987
Epoch 19: train loss: 0.6151972413063049
Epoch 20: train loss: 0.6123707294464111
Epoch 21: train loss: 0.60954749584198
Epoch 22: train loss: 0.6067266464233398
Epoch 23: train loss: 0.6039075255393982
Epoch 24: train loss: 0.601089

Epoch 197: train loss: 0.19557972252368927
Epoch 198: train loss: 0.19497253000736237
Epoch 199: train loss: 0.19437742233276367
Epoch 200: train loss: 0.19379407167434692
Epoch 201: train loss: 0.1932222694158554
Epoch 202: train loss: 0.19266174733638763
Epoch 203: train loss: 0.19211222231388092
Epoch 204: train loss: 0.19157347083091736
Epoch 205: train loss: 0.19104522466659546
Epoch 206: train loss: 0.1905273199081421
Epoch 207: train loss: 0.19001944363117218
Epoch 208: train loss: 0.18952137231826782
Epoch 209: train loss: 0.18903294205665588
Epoch 210: train loss: 0.18855386972427368
Epoch 211: train loss: 0.1880839765071869
Epoch 212: train loss: 0.1876230388879776
Epoch 213: train loss: 0.1871708631515503
Epoch 214: train loss: 0.18672728538513184
Epoch 215: train loss: 0.18629203736782074
Epoch 216: train loss: 0.18586498498916626
Epoch 217: train loss: 0.1854458749294281
Epoch 218: train loss: 0.1850346326828003
Epoch 219: train loss: 0.18463100492954254
Epoch 220: train l

Epoch 392: train loss: 0.1604309380054474
Epoch 393: train loss: 0.16038823127746582
Epoch 394: train loss: 0.1603458970785141
Epoch 395: train loss: 0.16030393540859222
Epoch 396: train loss: 0.1602623462677002
Epoch 397: train loss: 0.160221129655838
Epoch 398: train loss: 0.1601802408695221
Epoch 399: train loss: 0.16013973951339722
Epoch 400: train loss: 0.1600995659828186
Epoch 401: train loss: 0.16005975008010864
Epoch 402: train loss: 0.16002029180526733
Epoch 403: train loss: 0.1599811464548111
Epoch 404: train loss: 0.15994234383106232
Epoch 405: train loss: 0.159903883934021
Epoch 406: train loss: 0.15986573696136475
Epoch 407: train loss: 0.15982790291309357
Epoch 408: train loss: 0.15979039669036865
Epoch 409: train loss: 0.15975321829319
Epoch 410: train loss: 0.15971633791923523
Epoch 411: train loss: 0.15967975556850433
Epoch 412: train loss: 0.1596435010433197
Epoch 413: train loss: 0.15960754454135895
Epoch 414: train loss: 0.15957188606262207
Epoch 415: train loss: 0.

In [22]:
bat_emb = bat_at_ball.shape[1]
bowl_emb = bowl_at_ball.shape[1]
num_res = results.shape[1]

# this is a safe bound number, u can make the program a tiny bit quicker by making this number smaller
max_size = 270

all_game_ids = df["game_id"].unique()

each_game = {}

for game_id in all_game_ids:
    indexes = df["game_id"] == game_id
    num_in_go = sum(indexes)
    
    in_values = torch.cat([bat_at_ball[indexes], 
                           bowl_at_ball[indexes],
                           results[indexes]], 1).reshape(num_in_go, bat_emb+bowl_emb+num_res)
    
    batch_res = results[indexes]
    
    in_values = in_values[:max_size]
    batch_res = batch_res[:max_size]
    
    if in_values.shape[0] < max_size:
        cat_tens = torch.zeros(max_size - in_values.shape[0], in_values.shape[1])
        in_values = torch.cat([in_values, cat_tens])
        
        cat_tens = torch.zeros(max_size - batch_res.shape[0], batch_res.shape[1])
        batch_res = torch.cat([batch_res, cat_tens])
    
    each_game[game_id] = (in_values[:-1], batch_res[1:], pitch[indexes][0])




In [34]:
list_of_games = [game for game, pred, pitch in each_game.values()]
x_data = torch.stack(list_of_games)

list_of_result =[pred for game, pred, pitch in each_game.values()]
y_data = torch.stack(list_of_result)

list_of_pitch = [torch.FloatTensor(pitch) for game, pred, pitch in each_game.values()]
pitch_data = torch.stack(list_of_pitch)

In [35]:
NUM_PITCHES = int(next(iter(each_game.values()))[2].shape[0])

In [36]:
NUM_PITCHES

39

# Define RNN model

adapted from https://blog.floydhub.com/a-beginners-guide-on-recurrent-neural-networks-with-pytorch/


In [37]:
class Model(nn.Module):
    def __init__(self, bowl_dim, batt_dim, output_size, hidden_dim, n_layers, model_type):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.model_type = model_type

        #Defining the layers
        # RNN Layer
        
        if model_type == "rnn":
            self.rnn = nn.RNN(bowl_dim + batt_dim + output_size, hidden_dim, n_layers, batch_first=True)
        elif model_type == "lstm":
            self.rnn = nn.LSTM(bowl_dim + batt_dim + output_size, hidden_dim, n_layers, batch_first=True)
        elif model_type == "gru": 
            self.rnn = nn.GRU(bowl_dim + batt_dim + output_size, hidden_dim, n_layers, batch_first=True)
        else:
            #cant be fucked to implent special error
            print(1/0)
        
            
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
        
        self.sig = torch.nn.Sigmoid()
        

        self.pitch_transform = nn.Linear(NUM_PITCHES, self.n_layers * self.hidden_dim)
    
    def forward(self, x_data, pitch_data=None):
        # define input of RNN as concatination of both players and the runs vector
        # print(x.shape)
        
        batch_size = x_data.shape[0]

        if pitch_data is None: 
            # Initializing hidden state for first input using method defined below
            hidden = self.init_hidden(batch_size)
        else:
            hidden = self.init_hidden(batch_size, pitch = pitch_data)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x_data, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = self.fc(out)
        
        return self.sig(out)
    
    def init_hidden(self, batch_size, pitch=None):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        
        if pitch is None:
            hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        else:
            # if we are given a pitch:
            hidden = self.pitch_transform(pitch)
            
            # small chance this will kick an error (change to 1 if broken)
            hidden = hidden.reshape(pitch.shape[0],  self.n_layers, self.hidden_dim)
            hidden = hidden.transpose(0,1)
            # if we want to look at pitch embeddings we need to pull apart this layer
            
        if self.model_type == "rnn":
            return hidden
        elif self.model_type == "lstm":
            c0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
            return (hidden, c0)
        elif self.model_type == "gru":
            return hidden


In [45]:
# Instantiate the model with hyperparameters
model = Model(bowl_dim = 10, batt_dim=10, output_size=16, hidden_dim=100, n_layers=3, model_type="rnn")
# We'll also set the model to the device that we defined earlier (default is CPU)
model.to(device)

# Define hyperparameters

lr=0.01

# Define Loss, Optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [46]:
x_data.shape

torch.Size([100, 269, 36])

In [47]:
y_data.shape

torch.Size([100, 269, 16])

In [51]:
# to run without pitch data
model(x_data).shape

torch.Size([100, 269, 16])

In [52]:
# to run with pitch data
model(x_data, pitch_data).shape

torch.Size([100, 269, 16])

In [53]:
# Training Run
n_epochs = 1000
for epoch in range(1, n_epochs + 1):
         
    # RNN
    
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    #input_seq.to(device)
    pred = model(x_data)

    loss = criterion(pred, y_data)
    loss.backward(retain_graph=True) # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly
        
    if epoch%10 == 1:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 1/1000............. Loss: 0.2523
Epoch: 11/1000............. Loss: 0.2496
Epoch: 21/1000............. Loss: 0.2470
Epoch: 31/1000............. Loss: 0.2444
Epoch: 41/1000............. Loss: 0.2418
Epoch: 51/1000............. Loss: 0.2391
Epoch: 61/1000............. Loss: 0.2364


KeyboardInterrupt: 

In [24]:
model(x_data)[0,6,:]

tensor([0.7509, 0.1499, 0.0924, 0.0836, 0.1137, 0.0784, 0.0843, 0.0697, 0.0682,
        0.0825, 0.0819, 0.0762, 0.0703], grad_fn=<SliceBackward0>)

In [25]:
y_data[0,7,:]

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

# Things LP want to see

1) All models running (rnn, lstm, ...), (using pitch data, not using pitch data)

1.5) hyperparam tuning (embedding size, rnn size, epochs, (find best parameters for each model)

2) training loss vs time

3) test set accuracy prediciting unseen balls ina  game

3.5) accuracy at certain number of balls (20 balls 60 balls 115 balls, 200, 220)

4) see prediction of games the model hasnt seen


